Тут:
1. На сьогднішню дату перевіряємо, чи є репорт за поточний місяць. Якщо його ще не має <Response [204]>,
    віднімаємо по дню від сьогодньшної дати та перевіряємо, що повертає request.
2. Формуємо {date}, щоб вставити його у API та отримуємо робочий лінк.
3. З робочого лінку отримуємо репорт на останню дату у вигляді json
4. Отримаємо актуальний список банків з їхніми МФО та полем gr_bank,що є:
    "А" - Банки з державною часткою
    "B" - Банки іноземних банківських груп
    "E" - Банки з приватним капіталом
    "F" - n/a

In [ ]:
import requests
from datetime import timedelta
from datetime import date
from datetime import datetime

In [2]:
# get the last actual link from today 


def last_report(today):
    BASE_URL = 'https://bank.gov.ua/NBUStatService/v1/statdirectory/banksfinrep?date={date}&period=m&json'
    month = today.month
    year = today.year

    if month in range(10,11,12):
        date=f"{year}{month}01"
    else:
        date=f"{year}0{month}01"
            
    url = BASE_URL.format(date=date)
    r = requests.get(url)
    return r,url


In [29]:
today=datetime.today()

In [30]:
r,_=last_report(today)
r,today.month

(<Response [204]>, 1)

In [21]:
r,url=last_report(today)
while r.status_code !=200:
        today=today-timedelta(days=1)
        r,url=last_report(today)

In [31]:
# good advice for 'while','try' :
# https://stackoverflow.com/questions/36142149/python-run-try-again-after-exception-caught-and-worked-out/36142210
while True:
    try:
        r,url=last_report(today)
        while r.status_code !=200:
            today=today-timedelta(days=1)
            r,url=last_report(today)# try code
        break # quit the loop if successful
    except:
        print("No connection with API")# error handling



In [32]:
today,r

(datetime.datetime(2020, 8, 31, 2, 2), <Response [200]>)

In [33]:
url

'https://bank.gov.ua/NBUStatService/v1/statdirectory/banksfinrep?date=20200801&period=m&json'

In [34]:
r = requests.get(url)
source=r.json()

In [36]:
name_pair = [(i['fullname'],i['mfo'],i['gr_bank']) for i in source]

In [39]:
banks = set(name_pair)

In [42]:
type(list(banks))

list